In [1]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [2]:
# !pip install cleverhans==3.0.1 tensorflow==1.12.0

In [3]:
from cleverhans.attacks import FastGradientMethod
from torchvision import datasets, transforms
from adversarial import AdversarialExperiment
import numpy as np
import pandas as pd

In [0]:
# home_dir = '/content/gdrive/My Drive/Colab Notebooks/newlogic'
home_dir = '.'
BATCH_SIZE = 100

In [0]:
test_data = datasets.MNIST(home_dir + '/mnist', train=False, transform=transforms.ToTensor())

In [0]:
models = pd.read_csv(home_dir + '/output/mnist-models.csv')

In [0]:
attack_specs = [
    {'ord': 2, 'epsilon': 3},
    {'ord': np.inf, 'epsilon': 0.2},
    {'ord': np.inf, 'epsilon': 0.3},
]

In [0]:
results = []
for attack_spec in attack_specs:
    fgsm_params = {
        'ord': attack_spec['ord'], 
        'eps': attack_spec['epsilon'], 
        'clip_min': 0., 
        'clip_max': 1.
    }
    ex = AdversarialExperiment(FastGradientMethod, fgsm_params, test_data, 
                               batch_size=BATCH_SIZE)  
    for model_name in models.name:
        print(attack_spec)
        model_path = home_dir + '/output/' + model_name + '.pkl'
        accuracies = ex.evaluate_model(model_path)
        results.append({
            'model_name': model_name,
            'accuracies': accuracies,
            'max_probs': ex.extract_max_probs_on_perturbed_examples(),
            'attack_type': 'FGM',
            **attack_spec
        })

In [0]:
pd.DataFrame(results).to_json(home_dir + '/output/mnist-fgm-results.json')

In [0]:
# for attack_spec in attack_specs:
#     fgsm_params = {
#         'ord': attack_spec['ord'], 
#         'eps': attack_spec['epsilon'], 
#         'clip_min': 0., 
#         'clip_max': 1.
#     }
#     ex = AdversarialExperiment(FastGradientMethod, fgsm_params, test_data, 
#                                batch_size=BATCH_SIZE)
#     print(fgsm_params)
#     for model_name in models.name.iloc[-1:]:
#         print(model_name)
#         ex.evaluate_model(home_dir + '/output/' + model_name + '.pkl')